In [23]:
import pandas as pd
import re

In [6]:
#
# todo 장르별로 description의 감성 분석을 해보자
# todo 어떤 장르의 긍정 감성이 많은가? 같은 분포도

In [18]:
netflix_df = pd.read_csv('4.4ref/netflix_titles.csv', encoding ='utf8', sep = ',', engine = 'python')

# len(netflix_df) # 7787
netflix_df2 = netflix_df[['type', 'description']]

wrong_way = netflix_df2.groupby('type') # 이렇게 하면 안됨. 뒤에 count()붙이거나 하면 되긴 하는데 모든 데이터 그룹화는 x
wrong_way # <pandas.core.groupby.generic.DataFrameGroupBy object at 0x0000015352539010>

netflix_typed = netflix_df2.groupby('type').apply(lambda x: x)
netflix_typed

C:\Users\A\AppData\Local\Temp\ipykernel_17220\4264831729.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  netflix_typed = netflix_df2.groupby('type').apply(lambda x: x)


type                                        description
type                                                                    
Movie   1       Movie  After a devastating earthquake hits Mexico Cit...
        2       Movie  When an army recruit is found dead, his fellow...
        3       Movie  In a postapocalyptic world, rag-doll robots hi...
        4       Movie  A brilliant group of students become card-coun...
        6       Movie  After an awful accident, a couple admitted to ...
...               ...                                                ...
TV Show 7767  TV Show  Strong-willed, middle-class Kashaf and carefre...
        7775  TV Show  A quest for freedom and legendary treasure beg...
        7777  TV Show  While living alone in a spooky town, a young g...
        7779  TV Show  An assortment of talent takes the stage for a ...
        7785  TV Show  Dessert wizard Adriano Zumbo looks for the nex...

[7787 rows x 2 columns]

In [ ]:
# 결측치 제거
netflix_typed.info() # 정보확인
netflix_typed = netflix_typed[netflix_typed['description'].notnull()] # 결측치 제거 - 인데, 원래 없음

netflix_typed['type'].value_counts()
# Movie: 5377 / TV Show: 2410

In [36]:
# 쓸데 없는 문자 제거 후에, 타입별로 나누기
netflix_typed['description'] = netflix_typed['description'].apply(lambda x : re.sub(r"[^a-zA-Z]", " ", x))
netflix_typed

# movie / tv_show
netflix_movie = netflix_typed[netflix_typed['type'] == 'Movie'][['description']]
netflix_tvshow = netflix_typed[netflix_typed['type'] == 'TV Show'][['description']]
netflix_movie # 5377
netflix_tvshow # 2410

description
type                                                         
Movie 1     After a devastating earthquake hits Mexico Cit...
      2     When an army recruit is found dead  his fellow...
      3     In a postapocalyptic world  rag doll robots hi...
      4     A brilliant group of students become card coun...
      6     After an awful accident  a couple admitted to ...
...                                                       ...
      7781  Dragged from civilian life  a former superhero...
      7782  When Lebanon s Civil War deprives Zozo of his ...
      7783  A scrappy but poor boy worms his way into a ty...
      7784  In this documentary  South African rapper Nast...
      7786  This documentary delves into the mystique behi...

[5377 rows x 1 columns]

In [ ]:
# 아래 코드는 "한글"을 형태소 단위로 토큰화 시키는건데, 영어를 위한게 따로 필요함
# from konlpy.tag import Okt
# okt = Okt()
# def okt_tokenizer(text):
#     tokens = okt.morphs(text)
#     return tokens
import nltk
from nltk.tokenize import word_tokenize
# nltk.download('punkt') # 최초 1번만 다운

# 영어 문장을 토큰화하기 위한 함수를 정의
def nltk_tokenizer(text):
    tokens = word_tokenize(text)
    return tokens

['After', 'aa', 'devastating', 'earthquake', 'hits', 'Mexico']

In [48]:
# 분석 모델 구축
# ? 분석 모델을 2개 구축하는 것에 의미가 있나 -> 이것을 how 활용?


from sklearn.feature_extraction.text import TfidfVectorizer

# 1. movie에 대한 분석 모델
tfidf_movie = TfidfVectorizer(tokenizer=nltk_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_movie.fit(netflix_movie['description'])
tfidf_movie_tfidf = tfidf_movie.transform(netflix_movie['description'])
tfidf_movie_tfidf

# 2. tv_show에 대한 분석 모델
tfidf_tvshow = TfidfVectorizer(tokenizer=nltk_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_tvshow.fit(netflix_tvshow['description'])
tfidf_tvshow_tfidf = tfidf_tvshow.transform(netflix_tvshow['description'])

# ? 된건가? -> print(tfidf_tvshow_tfidf) 찍어보니까, (~,0 : ~~~) 이렇게 값이 영단어가 아니라 0으로만나오는데?

In [49]:
# 감성분류모델 구축
from sklearn.linear_model import LogisticRegression

SA_lr = LogisticRegression(random_state=0)

SA_lr.fit(tfidf_movie_tfidf, netflix_movie['description'])
# SA_lr.fit(tfidf_tvshow_tfidf, netflix_tvshow['description'])

# ---
from sklearn.model_selection import GridSearchCV

params = {'C': [1,3,3.5,4,4.5,5]}
SA_lr_grid_cv = GridSearchCV(SA_lr, param_grid=params, cv=3, scoring='accuracy', verbose=1)

SA_lr_grid_cv.fit(tfidf_movie_tfidf, netflix_movie['description'])
# SA_lr_grid_cv.fit(tfidf_tvshow_tfidf, netflix_tvshow['description'])

# GridSearchCV에 의해 찾은 최적의 C 매개변수best_params와 최고 점수best_score를 출력하여 확인
print(SA_lr_grid_cv.best_params_, round(SA_lr_grid_cv.best_score_, 4))

# 최적 매개변수가 설정된 모형best_estimator을 SA_lr_best 객체에 저장
SA_lr_best = SA_lr_grid_cv.best_estimator_ # #최적 매개변수의 best 모델 저장

Fitting 3 folds for each of 6 candidates, totalling 18 fits


C:\Users\A\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
# 1)아니 근데, 애초에 description에 감성 분류(0,1) label이 되어있어야 가능함
# -> 아예 했던 것을 전혀 이해하지 못하고, 복붙만 하면서 진행했음
# -> 폐기폐기;;
# -> 2)이전에 했던 모델(8-1.ipynb)로 하려고 해도, "한글" 감성분석 모델인데 "영어" 텍스트를 분석할 수 없다
# *어이고; 다시 back..